In [110]:
def to_signed_16bit_hex(n):
    """
    Convert an integer n (−32768 ≤ n < 32768) into its
    16-bit two’s-complement hex representation (4 digits).
    """
    if not -2**15 <= n < 2**15:
        raise ValueError("Value out of range for signed 16-bit")
    # mask to 16 bits and format as upper-case hex with 4 digits
    return format(n & 0xFFFF, '04X')

In [21]:
import pickle
import numpy as np

with open('groups.pkl', 'rb') as f:
    data = pickle.load(f)
src_ids = data['SrcID_filt']
det_ids = data['DetID_filt']
src_num = data['src_num']
x = np.array(src_ids)
print(np.where(x==202039301010095)[0])
src_ids_cut = src_ids
det_ids_cut = det_ids
src_num_cut = src_num
print(src_num_cut[0])
src_ids_cut[0]
print(src_num_cut[5524])
print(len(src_num_cut))

[5524]
6
167
7152


In [112]:
def download_files(srcid, file_pattern, output_dir,id_list):
    """
    Download only the PPS files matching file_pattern for a given XMM SRCID.
    Returns as before, but will try all OBS_IDs until it finds wanted files.
    """
    from astroquery.heasarc import Heasarc
    import numpy as np
    import os
    import fnmatch
    import requests
    from bs4 import BeautifulSoup
    from urllib.parse import urljoin
    from requests.exceptions import HTTPError

    heasarc = Heasarc()
    query  = f"SELECT * FROM xmmssc WHERE SRCID = '{srcid}'"
    tbl    = heasarc.query_tap(query).to_table()

    if len(tbl) == 0:
        raise RuntimeError(f"No xmmssc entry found for SRCID={srcid}")

    camera = [0, 0, 0]  # PN, M1, M2
    sr_filename = ["", "", ""]
    found_obsid = None
    file_urls = []

    # Try all matching obsids until files are found
    for row in tbl:
        obsid = row['obsid'].strip()
        pps_url = f"https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/{obsid}/PPS/"
        try:
            r = requests.get(pps_url, timeout=10)
            r.raise_for_status()
        except HTTPError:
            print(f"[{srcid}] OBS_ID {obsid} not found (404) — checking next.")
            continue
        except Exception as e:
            print(f"[{srcid}] OBS_ID {obsid} failed with error: {e}")
            continue

        # Step 2: Scrape the PPS directory for matching files
        print(f"[{srcid}] Scanning OBS_ID {obsid} PPS directory {pps_url}")
        soup = BeautifulSoup(r.text, "html.parser")
        file_urls = []
        camera = [0, 0, 0]
        sr_filename = ["", "", ""]
        for a in soup.find_all("a", href=True):
            name = a.get_text(strip=True)
            if fnmatch.fnmatch(name, file_pattern):
                url = urljoin(pps_url, a["href"])
                if "SR" in file_pattern:
                    if "PN" in name:
                        camera[0] = 1
                        sr_filename[0] = name
                    if "M1" in name:
                        camera[1] = 1
                        sr_filename[1] = name
                    if "M2" in name:
                        camera[2] = 1
                        sr_filename[2] = name
                file_urls.append(url)

        if file_urls:
            found_obsid = obsid
            break  # Stop at first OBS_ID with desired files

    if not file_urls:
        print(f"[{srcid}] No files matching {file_pattern} found in any PPS directory.")
        id_list.append(srcid)
        return camera, sr_filename, None, id_list

    # Step 3: Download files (skip if already exist)
    os.makedirs(output_dir, exist_ok=True)
    print(f"[{srcid}] Downloading {len(file_urls)} file(s) from OBS_ID {found_obsid}:")

    for url in file_urls:
        fn = os.path.basename(url)
        dest = os.path.join(output_dir, fn)
        if os.path.exists(dest):
            print(f"[{srcid}] [SKIP] {fn} already exists.")
            continue
        print(f"[{srcid}] → Downloading {fn}")
        rr = requests.get(url, stream=True)
        rr.raise_for_status()
        with open(dest, "wb") as f:
            for chunk in rr.iter_content(8192):
                f.write(chunk)

    print(f"[{srcid}] Done. Files in: {output_dir}\n")
    return camera, sr_filename, found_obsid, id_list

In [113]:
import shutil
from pathlib import Path
import os
def copy_file_to_folder(src_path, dest_folder):
    """
    Copy a file at src_path into dest_folder, preserving the filename.
    Creates dest_folder if it doesn’t already exist.
    """
    src = Path(src_path).expanduser()
    if not src.is_file():
        raise FileNotFoundError(f"Source file not found: {src}")

    dest_dir = Path(dest_folder).expanduser()
    dest_dir.mkdir(parents=True, exist_ok=True)

    dest = dest_dir / src.name
    shutil.copy2(src, dest)   # copy2 preserves metadata; use shutil.copy for just content
    print(f"Copied {src} → {dest}")
    return dest

'''if __name__ == "__main__":
    filename = "example.txt"
    # suppose you found this in ~/Downloads/storage/
    source_path = os.path.expanduser(f"~/Downloads/storage/{filename}")
    # you want it in ~/Downloads/files/
    destination_folder = "~/Downloads/files"
'''


'if __name__ == "__main__":\n    filename = "example.txt"\n    # suppose you found this in ~/Downloads/storage/\n    source_path = os.path.expanduser(f"~/Downloads/storage/{filename}")\n    # you want it in ~/Downloads/files/\n    destination_folder = "~/Downloads/files"\n'

In [131]:
import os
from astropy.io import fits

camera = []
name = [""]
id_list = []
id_bg_list = []
id_arf_list = []
#SrcID_filt = [src_ids[57]]
for i, src_id in enumerate(src_ids_cut):
    id_num = int(str(src_num_cut[i]))
    id_hex = to_signed_16bit_hex(id_num)
    src_wildcard = "*SRSPEC*" + id_hex + ".FTZ"
    bg_wildcard = "*BGSPEC*" + id_hex + ".FTZ"
    arf_wildcard = "*SRCARF*" + id_hex + ".FTZ"
    folder_name = "~/benjaminL/XMM_Analysis/test_script/" + str(src_id)
    camera, name, obsid, id_list = download_files(str(src_id), src_wildcard,os.path.expanduser(folder_name),id_list)
    download_files(str(src_id), bg_wildcard,os.path.expanduser(folder_name), id_bg_list)
    download_files(str(src_id), arf_wildcard,os.path.expanduser(folder_name), id_arf_list)
    
    if camera[0]==1:
        print(folder_name)
        print(name[0])
        fits_path = folder_name + "/" + name[0]
        with fits.open(fits_path) as hdul:
            spec_hdr = hdul["SPECTRUM"].header
        copy_file_to_folder("~/benjaminL/XMM_Analysis/rmf_reference/" + spec_hdr.get("RESPFILE"), folder_name)
    if camera[1]==1:
        print(folder_name)
        print(name[1])
        fits_path = folder_name + "/" + name[1]
        print(fits_path)
        with fits.open(fits_path) as hdul:
            spec_hdr = hdul["SPECTRUM"].header
        copy_file_to_folder("~/benjaminL/XMM_Analysis/rmf_reference/" + spec_hdr.get("RESPFILE"), folder_name)
    if camera[2]==1:
        print(folder_name)
        print(name[0])
        fits_path = folder_name + "/" + name[2]
        with fits.open(fits_path) as hdul:
            spec_hdr = hdul["SPECTRUM"].header
        copy_file_to_folder("~/benjaminL/XMM_Analysis/rmf_reference/" + spec_hdr.get("RESPFILE"), folder_name)
#Flag: 209167908010002, 209167908010003,209167908010001 all same obs no xmm pps folder 202026708015352

[201129702010064] Scanning OBS_ID 0802410101 PPS directory https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0802410101/PPS/
[201129702010064] Scanning OBS_ID 0694641401 PPS directory https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0694641401/PPS/
[201129702010064] Scanning OBS_ID 0144220101 PPS directory https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0144220101/PPS/
[201129702010064] Scanning OBS_ID 0862471201 PPS directory https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0862471201/PPS/
[201129702010064] Scanning OBS_ID 0112970201 PPS directory https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0112970201/PPS/
[201129702010064] No files matching *SRSPEC004E.FTZ found in any PPS directory.
[201129702010064] Scanning OBS_ID 0802410101 PPS directory https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0802410101/PPS/
[201129702010064] Scanning OBS_ID 0694641401 PPS directory https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0694641401/PPS/
[201129702010064] Scanning OBS_ID 0144220101 PPS directory 

In [102]:
id_list

['208024101010269', '201129702010064', '201442201010054', '202039301010075']

In [ ]:
import os
import shutil

def delete_srcid_folders(base_dir, srcid_list):
    """
    Deletes folders within base_dir named after the items in srcid_list.

    Parameters:
        base_dir (str): Directory containing srcid folders.
        srcid_list (list): List of srcid numbers (as integer or string).
    """
    for srcid in srcid_list:
        folder_path = os.path.join(base_dir, str(srcid))
        if os.path.isdir(folder_path):
            shutil.rmtree(folder_path)
            print(f"Deleted: {folder_path}")
        else:
            print(f"Not found: {folder_path}")

# Example Usage:
base_directory = os.path.expanduser('~/benjaminL/XMM_Analysis/test_script')
srcid_numbers = src_ids_cut
delete_srcid_folders(base_directory, srcid_numbers)

In [ ]:
import os

def delete_files_with_substring(base_dir, srcid_list, substring):
    """
    Deletes files containing a specific substring in folders named after srcids.

    Parameters:
        base_dir (str): Directory containing srcid folders.
        srcid_list (list): List of srcid numbers (as integer or string).
        substring (str): Substring to search for in filenames.
    """
    for srcid in srcid_list:
        folder_path = os.path.join(base_dir, str(srcid))
        if os.path.isdir(folder_path):
            found = False
            for fname in os.listdir(folder_path):
                if substring in fname:
                    file_path = os.path.join(folder_path, fname)
                    os.remove(file_path)
                    found = True
                    print(f"Deleted: {file_path}")
            if not found:
                print(f"No files matching '{substring}' in: {folder_path}")
        else:
            print(f"Not found: {folder_path}")

# Example Usage:
base_directory = os.path.expanduser('~/benjaminL/XMM_Analysis/test_script')
srcid_numbers = src_ids_cut
substring_to_delete = "EVLI"  # <-- replace with your real substring
delete_files_with_substring(base_directory, srcid_numbers, substring_to_delete)

Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/206946410010071/P0862470401PNS003PIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/205547504010047/P0554750401PNS003PIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/201442201010054/P0802410101PNS003PIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/201442201010054/P0802410101M1S001MIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/201442201010054/P0802410101M2S002MIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/207848601010131/P0784860101PNS003PIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/208016815010025/P0801681501PNS003PIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/209322007010003/P0932200701M2S002MIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/209322007010003/P0932200701PNS003PIEVLI0000.FTZ
Not found: /Users/students/b

Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/201645614010034/P0930590101PNS001PIEVLI0000.FTZ
Not found: /Users/students/benjaminL/XMM_Analysis/test_script/202026707010263
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/201529201010017/P0152920101PNS003PIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/201529201010017/P0152920101M1S001MIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/201529201010017/P0152920101M2S002MIEVLI0000.FTZ
Not found: /Users/students/benjaminL/XMM_Analysis/test_script/206735502010100
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/203028842010020/P0762250301PNS002PIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/203028842010020/P0762250301M1S001MIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/203028842010020/P0762250301M2S005MIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/207436101010151/P0743610101P

Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/208402111010010/P0840211101PNS003PIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/208860210010005/P0886021701PNS003PIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/208860210010005/P0886021701M2S002MIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/208860210010005/P0886021701M1S001MIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/208860307010001/P0886030701M1S001MIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/208860307010001/P0886030701M2S002MIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/208860307010001/P0886030701PNS003PIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/208400403010036/P0840040301PNS003PIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/208860202010005/P0886020201PNS003PIEVLI0000.FTZ
Deleted: /Users/students/ben

Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/209322007010008/P0932200701PNS003PIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/208016820010014/P0801682001PNS003PIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/206514501010009/P0651450101M2S002MIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/208861106010006/P0886110601PNS003PIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/208861103010004/P0886110301PNS003PIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/201482104010023/P0554600301M2U002MIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/201482104010023/P0554600301PNU002PIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/201482104010023/P0554600301M1U002MIEVLI0000.FTZ
Not found: /Users/students/benjaminL/XMM_Analysis/test_script/207622503015289
Deleted: /Users/students/benjaminL/XMM_Analysis/test_scrip

Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/207446001010022/P0744600101M2S002MIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/207446001010022/P0744600101M2S002TIEVLI0000.FTZ
Not found: /Users/students/benjaminL/XMM_Analysis/test_script/206946406010142
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/207234103010119/P0723410301PNS003PIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/207234103010119/P0723410301M1S001MIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/207234103010119/P0723410301M2S002MIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/207436101010168/P0743610101M1S001MIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/201529201010020/P0400340101PNS003PIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/201129707010063/P0851180901PNS003PIEVLI0000.FTZ
Not found: /Users/students/benjaminL/XMM_Analysis/test_scr

...

Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/201129721010028/P0658600101M2S003MIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/203028845010026/P0862470201PNS003PIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/208301904010274/P0830190401M1S001MIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/208301904010274/P0830190401M2S002MIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/208301904010274/P0830190401M2S002TIEVLI0000.FTZ
Not found: /Users/students/benjaminL/XMM_Analysis/test_script/205538501010018
Not found: /Users/students/benjaminL/XMM_Analysis/test_script/207436101010172
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/208860709010055/P0886070901M1S001MIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/208860709010096/P0886070901PNS003PIEVLI0000.FTZ
Not found: /Users/students/benjaminL/XMM_Analysis/test_script/208860705010056
Deleted: /

Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/206541901010006/P0701230701M1S002MIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/208400403010051/P0886030301PNS003PIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/208860207010026/P0886020701PNS003PIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/208230301010001/P0823030101M2S002MIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/208230301010001/P0823030101M1S001MIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/208400403010045/P0840040301PNS003PIEVLI0000.FTZ
Not found: /Users/students/benjaminL/XMM_Analysis/test_script/207622503010311
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/201113503010039/P0506291201M2S002MIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/201113503010039/P0506291201M1S001MIEVLI0000.FTZ
Not found: /Users/students/benjaminL/XMM_Analysis/test_scr

Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/201129705010021/P0893811101M1S001MIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/202039301010018/P0862471101PNS003PIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/201129705010090/P0694640401PNS003PIEVLI0000.FTZ
Not found: /Users/students/benjaminL/XMM_Analysis/test_script/201480905010013
Not found: /Users/students/benjaminL/XMM_Analysis/test_script/206929601015085
Not found: /Users/students/benjaminL/XMM_Analysis/test_script/209168106010020
Not found: /Users/students/benjaminL/XMM_Analysis/test_script/209168109010019
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/208860310010003/P0886031001PNS003PIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/208860310010003/P0886031001M2U002MIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/208860210010034/P0886021001M2U002MIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_A

Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/203028825010029/P0694641301M1S001MIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/208860802010007/P0886080201PNS003PIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/208860709010011/P0886070901M2S002MIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/208860709010011/P0886070901M1S001MIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/208860709010011/P0886070901PNS003PIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/208860603010002/P0886060301PNS003PIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/209168112010006/P0916811201M1S001MIEVLI0000.FTZ
Not found: /Users/students/benjaminL/XMM_Analysis/test_script/209167911010010
Not found: /Users/students/benjaminL/XMM_Analysis/test_script/205538501010004
Not found: /Users/students/benjaminL/XMM_Analysis/test_script/208860304010006
Deleted: /

Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/202039301010010/P0203930101PNS003PIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/201129715010004/P0802410101PNS003PIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/201129715010004/P0802410101M1S001MIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/201129715010004/P0802410101M2S002MIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/202039301010093/P0802410101PNS003PIEVLI0000.FTZ
Not found: /Users/students/benjaminL/XMM_Analysis/test_script/206946413010187
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/208016815010036/P0801681501PNS003PIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/201129701010041/P0694640801PNS003PIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_script/208016815010022/P0801681501PNS003PIEVLI0000.FTZ
Deleted: /Users/students/benjaminL/XMM_Analysis/test_scrip

In [23]:
from astroquery.heasarc import Heasarc
import numpy as np
import os
import fnmatch
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from requests.exceptions import HTTPError

def download_events_files(srcid, file_pattern1, output_dir):
    """
    Download only the PPS files matching file_pattern for a given XMM SRCID.

    Parameters
    ----------
    srcid : str
        The XMM_SRCID to query (e.g. "208860601010006").
    file_pattern : str
        Shell-style wildcard for the filenames you want (e.g. "*SRSPEC0002.FTZ").
    output_dir : str
        Path to the folder where the matched files will be saved.

    Returns
    -------
    camera : list of int
        Flags [PN, M1, M2] indicating which cameras were found.
    sr_filename : list of str
        Filenames for each camera index, or empty string if missing.
    """
    heasarc = Heasarc()
    
    query  = f"SELECT * FROM xmmssc WHERE SRCID = '{srcid}'"
    tbl    = heasarc.query_tap(query).to_table()

    if len(tbl) == 0:
        raise RuntimeError(f"No xmmssc entry found for SRCID={srcid}")

    found_obsid = None

    # Step 1: Try all matching obsids and find the first with a valid PPS directory
    for row in tbl:
        obsid = row['obsid'].strip()
        pps_url = f"https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/{obsid}/PPS/"
        try:
            r = requests.get(pps_url, timeout=10)
            r.raise_for_status()
            found_obsid = obsid
            print(f"[{srcid}] Using OBS_ID {obsid} → PPS directory: {pps_url}")
            break  # Use this one
        except HTTPError:
            print(f"[{srcid}] OBS_ID {obsid} not found (404) — checking next.")
        except Exception as e:
            print(f"[{srcid}] OBS_ID {obsid} failed with error: {e}")

    if found_obsid is None:
        raise RuntimeError(f"[{srcid}] No valid PPS directory found for any matching OBS_ID.")

    # Step 2: Scrape the PPS directory for matching files
    pps_url = f"https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/{found_obsid}/PPS/"
    r = requests.get(pps_url)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")

    file_urls = []
   
    
    for a in soup.find_all("a", href=True):
        name = a.get_text(strip=True)
        if fnmatch.fnmatch(name, file_pattern1):
            url = urljoin(pps_url, a["href"])
            file_urls.append(url)

    if not file_urls:
        print(f"[{srcid}] No files matching {file_pattern1} found.")
        
    # Step 3: Download files (skip if already exist)
    os.makedirs(output_dir, exist_ok=True)
    print(f"[{srcid}] Downloading {len(file_urls)} file(s):")

    for url in file_urls:
        fn = os.path.basename(url)
        dest = os.path.join(output_dir, fn)
        if os.path.exists(dest):
            print(f"[{srcid}] [SKIP] {fn} already exists.")
            continue
        print(f"[{srcid}] → Downloading {fn}")
        rr = requests.get(url, stream=True)
        rr.raise_for_status()
        with open(dest, "wb") as f:
            for chunk in rr.iter_content(8192):
                f.write(chunk)

    print(f"[{srcid}] Done. Files in: {output_dir}\n")

In [21]:
from astroquery.heasarc import Heasarc
import numpy as np
import os
import fnmatch
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from requests.exceptions import HTTPError

def check_files(srcid, file_pattern, output_dir):
    """
    Download only the PPS files matching file_pattern for a given XMM SRCID.

    Parameters
    ----------
    srcid : str
        The XMM_SRCID to query (e.g. "208860601010006").
    file_pattern : str
        Shell-style wildcard for the filenames you want (e.g. "*SRSPEC0002.FTZ").
    output_dir : str
        Path to the folder where the matched files will be saved.

    Returns
    -------
    camera : list of int
        Flags [PN, M1, M2] indicating which cameras were found.
    sr_filename : list of str
        Filenames for each camera index, or empty string if missing.
    """
    heasarc = Heasarc()
    
    query  = f"SELECT * FROM xmmssc WHERE SRCID = '{srcid}'"
    tbl    = heasarc.query_tap(query).to_table()

    if len(tbl) == 0:
        raise RuntimeError(f"No xmmssc entry found for SRCID={srcid}")

    found_obsid = None

    # Step 1: Try all matching obsids and find the first with a valid PPS directory
    for row in tbl:
        obsid = row['obsid'].strip()
        pps_url = f"https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/{obsid}/PPS/"
        try:
            r = requests.get(pps_url, timeout=10)
            r.raise_for_status()
            found_obsid = obsid
            print(f"[{srcid}] Using OBS_ID {obsid} → PPS directory: {pps_url}")
            break  # Use this one
        except HTTPError:
            print(f"[{srcid}] OBS_ID {obsid} not found (404) — checking next.")
        except Exception as e:
            print(f"[{srcid}] OBS_ID {obsid} failed with error: {e}")

    if found_obsid is None:
        raise RuntimeError(f"[{srcid}] No valid PPS directory found for any matching OBS_ID.")

    # Step 2: Scrape the PPS directory for matching files
    pps_url = f"https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/{found_obsid}/PPS/"
    r = requests.get(pps_url)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")

    file_urls = []
    camera = [0, 0, 0]  # PN, M1, M2
    
    for a in soup.find_all("a", href=True):
        name = a.get_text(strip=True)
        if fnmatch.fnmatch(name, file_pattern):
            url = urljoin(pps_url, a["href"])
            if "SR" in file_pattern:
                if "PN" in name:
                    camera[0] = 1
                if "M1" in name:
                    camera[1] = 1
                if "M2" in name:
                    camera[2] = 1
            file_urls.append(url)

    if not file_urls:
        print(f"[{srcid}] No files matching {file_pattern} found.")
    
    return camera, found_obsid
    

In [ ]:
import os
from astropy.io import fits

camera = []
name = [""]
SrcID_filt = src_ids_cut
for src_id in SrcID_filt:
    id_num = int(str(src_id)[-4:])
    id_hex = to_signed_16bit_hex(id_num)
    src_wildcard = "*SRSPEC" + id_hex + ".FTZ"
    folder_name = "~/benjaminL/XMM_Analysis/test_script/" + str(src_id)
    camera, obsid = check_files(str(src_id), src_wildcard,os.path.expanduser(folder_name))
    
    if camera[0]==1:
        event_wildcard = "P" + str(obsid)+ "PN*EVLI*"
        download_events_files(str(src_id),event_wildcard,os.path.expanduser(folder_name))
    if camera[1]==1:
        event_wildcard = "P" + str(obsid)+ "M1*EVLI*" 
        download_events_files(str(src_id),event_wildcard,os.path.expanduser(folder_name))
    if camera[2]==1:
        event_wildcard = "P" + str(obsid)+ "M2*EVLI*"
        download_events_files(str(src_id),event_wildcard,os.path.expanduser(folder_name))
        
#Flag: 209167908010002, 209167908010003,209167908010001 all same obs no xmm pps folder
#STOPPED: 201529201010020

[208860311010003] Done. Files in: /Users/students/benjaminL/XMM_Analysis/test_script/208860311010003

[208860311010003] Using OBS_ID 0886031101 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0886031101/PPS/
[208860311010003] Downloading 1 file(s):
[208860311010003] → Downloading P0886031101M1S001MIEVLI0000.FTZ
[208860311010003] Done. Files in: /Users/students/benjaminL/XMM_Analysis/test_script/208860311010003

[208860311010003] Using OBS_ID 0886031101 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0886031101/PPS/
[208860311010003] Downloading 1 file(s):
[208860311010003] → Downloading P0886031101M2S002MIEVLI0000.FTZ
[208860311010003] Done. Files in: /Users/students/benjaminL/XMM_Analysis/test_script/208860311010003

[209321909010002] Using OBS_ID 0932191101 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0932191101/PPS/
[209321909010002] No files matching *SRSPEC0002.FTZ found.
[208860904010005] Using OBS_ID 0886090601 → PPS director

[208860703010005] Using OBS_ID 0886070301 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0886070301/PPS/
[208860703010005] Using OBS_ID 0886070301 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0886070301/PPS/
[208860703010005] Downloading 1 file(s):
[208860703010005] → Downloading P0886070301PNU002PIEVLI0000.FTZ
[208860703010005] Done. Files in: /Users/students/benjaminL/XMM_Analysis/test_script/208860703010005

[208860611010004] Using OBS_ID 0886070301 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0886070301/PPS/
[208860611010004] Using OBS_ID 0886070301 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0886070301/PPS/
[208860611010004] Downloading 1 file(s):
[208860611010004] → Downloading P0886070301PNU002PIEVLI0000.FTZ
[208860611010004] Done. Files in: /Users/students/benjaminL/XMM_Analysis/test_script/208860611010004

[208860602010004] Using OBS_ID 0886060201 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/x

[204065802010003] Using OBS_ID 0748391201 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0748391201/PPS/
[204065802010003] Using OBS_ID 0748391201 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0748391201/PPS/
[204065802010003] Downloading 1 file(s):
[204065802010003] → Downloading P0748391201M1S002MIEVLI0000.FTZ
[204065802010003] Done. Files in: /Users/students/benjaminL/XMM_Analysis/test_script/204065802010003

[204065802010003] Using OBS_ID 0748391201 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0748391201/PPS/
[204065802010003] Downloading 1 file(s):
[204065802010003] → Downloading P0748391201M2S003MIEVLI0000.FTZ
[204065802010003] Done. Files in: /Users/students/benjaminL/XMM_Analysis/test_script/204065802010003

[207641901010002] Using OBS_ID 0764190101 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0764190101/PPS/
[207641901010002] Using OBS_ID 0764190101 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/x

[202026707010284] No files matching *SRSPEC011C.FTZ found.
[206946405010092] Using OBS_ID 0694640501 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0694640501/PPS/
[206946405010092] No files matching *SRSPEC005C.FTZ found.
[201129708010046] Using OBS_ID 0673550201 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0673550201/PPS/
[201129708010046] Using OBS_ID 0673550201 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0673550201/PPS/
[201129708010046] Downloading 2 file(s):
[201129708010046] → Downloading P0673550201M2S002MIEVLI0000.FTZ
[201129708010046] → Downloading P0673550201M2S002TIEVLI0000.FTZ
[201129708010046] Done. Files in: /Users/students/benjaminL/XMM_Analysis/test_script/201129708010046

[206556701010028] Using OBS_ID 0655670101 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0655670101/PPS/
[206556701010028] Using OBS_ID 0655670101 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0655670101/P

[208860702010005] Using OBS_ID 0886070201 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0886070201/PPS/
[208860702010005] Using OBS_ID 0886070201 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0886070201/PPS/
[208860702010005] Downloading 1 file(s):
[208860702010005] → Downloading P0886070201PNS003PIEVLI0000.FTZ
[208860702010005] Done. Files in: /Users/students/benjaminL/XMM_Analysis/test_script/208860702010005

[208860702010005] Using OBS_ID 0886070201 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0886070201/PPS/
[208860702010005] Downloading 1 file(s):
[208860702010005] → Downloading P0886070201M2S002MIEVLI0000.FTZ
[208860702010005] Done. Files in: /Users/students/benjaminL/XMM_Analysis/test_script/208860702010005

[201032612010008] Using OBS_ID 0103261201 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0103261201/PPS/
[201032612010008] No files matching *SRSPEC0008.FTZ found.
[208860404010037] Using OBS_ID 0886

...

[201129702010004] Using OBS_ID 0144220101 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0144220101/PPS/
[201129702010004] Downloading 1 file(s):
[201129702010004] → Downloading P0144220101M1U002MIEVLI0000.FTZ
[201129702010004] Done. Files in: /Users/students/benjaminL/XMM_Analysis/test_script/201129702010004

[201129702010004] Using OBS_ID 0144220101 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0144220101/PPS/
[201129702010004] Downloading 1 file(s):
[201129702010004] → Downloading P0144220101M2U002MIEVLI0000.FTZ
[201129702010004] Done. Files in: /Users/students/benjaminL/XMM_Analysis/test_script/201129702010004

[203028832010004] Using OBS_ID 0694640401 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0694640401/PPS/
[203028832010004] Using OBS_ID 0694640401 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0694640401/PPS/
[203028832010004] Downloading 1 file(s):
[203028832010004] → Downloading P0694640401PNS003PIEV

[201129705010028] Downloading 1 file(s):
[201129705010028] → Downloading P0862470701M1S001MIEVLI0000.FTZ
[201129705010028] Done. Files in: /Users/students/benjaminL/XMM_Analysis/test_script/201129705010028

[201129705010028] Using OBS_ID 0862470701 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0862470701/PPS/
[201129705010028] Downloading 1 file(s):
[201129705010028] → Downloading P0862470701M2S002MIEVLI0000.FTZ
[201129705010028] Done. Files in: /Users/students/benjaminL/XMM_Analysis/test_script/201129705010028

[201129705015064] Using OBS_ID 0723410501 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0723410501/PPS/
[201129705015064] No files matching *SRSPEC13C8.FTZ found.
[201129705010021] Using OBS_ID 0893811101 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0893811101/PPS/
[201129705010021] Using OBS_ID 0893811101 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0893811101/PPS/
[201129705010021] Downloading 1 fil

[208400403010043] Using OBS_ID 0886030301 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0886030301/PPS/
[208400403010043] Downloading 1 file(s):
[208400403010043] → Downloading P0886030301PNS003PIEVLI0000.FTZ
[208400403010043] Done. Files in: /Users/students/benjaminL/XMM_Analysis/test_script/208400403010043

[208400403010186] Using OBS_ID 0840040301 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0840040301/PPS/
[208400403010186] No files matching *SRSPEC00BA.FTZ found.
[207641910010001] Using OBS_ID 0764191001 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0764191001/PPS/
[207641910010001] Using OBS_ID 0764191001 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0764191001/PPS/
[207641910010001] Downloading 1 file(s):
[207641910010001] → Downloading P0764191001PNS003PIEVLI0000.FTZ
[207641910010001] Done. Files in: /Users/students/benjaminL/XMM_Analysis/test_script/207641910010001

[207641910010001] Using OBS_ID 0764

[206946408010015] Downloading 1 file(s):
[206946408010015] → Downloading P0783160101PNS003PIEVLI0000.FTZ
[206946408010015] Done. Files in: /Users/students/benjaminL/XMM_Analysis/test_script/206946408010015

[206946408010015] Using OBS_ID 0783160101 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0783160101/PPS/
[206946408010015] Downloading 1 file(s):
[206946408010015] → Downloading P0783160101M2S002MIEVLI0000.FTZ
[206946408010015] Done. Files in: /Users/students/benjaminL/XMM_Analysis/test_script/206946408010015

[201113501015104] Using OBS_ID 0862470501 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0862470501/PPS/
[201113501015104] No files matching *SRSPEC13F0.FTZ found.
[201129706010007] Using OBS_ID 0694640201 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0694640201/PPS/
[201129706010007] Using OBS_ID 0694640201 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0694640201/PPS/
[201129706010007] Downloading 1 fil

[207641909010004] Done. Files in: /Users/students/benjaminL/XMM_Analysis/test_script/207641909010004

[205056701010169] Using OBS_ID 0505670101 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0505670101/PPS/
[205056701010169] Using OBS_ID 0505670101 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0505670101/PPS/
[205056701010169] Downloading 1 file(s):
[205056701010169] → Downloading P0505670101PNU002PIEVLI0000.FTZ
[205056701010169] Done. Files in: /Users/students/benjaminL/XMM_Analysis/test_script/205056701010169

[207234105010220] Using OBS_ID 0723410501 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0723410501/PPS/
[207234105010220] Using OBS_ID 0723410501 → PPS directory: https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0723410501/PPS/
[207234105010220] Downloading 1 file(s):
[207234105010220] → Downloading P0723410501M1S001MIEVLI0000.FTZ
[207234105010220] Done. Files in: /Users/students/benjaminL/XMM_Analysis/test_script/207234105